In [1]:
import re 
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
import requests 
from rasterio.plot import show
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
import json
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
''' For getting the lambert co-ordinates from given address from geopunt API within the Flanders'''

def for_coordinates(address):
    try:
        req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1",)
        x = req.json()["LocationResult"][0]["Location"]["X_Lambert72"]
        y = req.json()["LocationResult"][0]["Location"]["Y_Lambert72"]
        return x, y
    
    except:
        print('Enter correct address')





In [3]:
''' For polygon co-ordinates for the given address '''

def polygon(address):
    
        address_regx = re.compile("^([A-z-]+)\s(\d+),\s(\d+)\s([A-z]+)") 
        result = address_regx.search(address)
        street = result.group(1)
        nb = result.group(2)
        pc = result.group(3)
        city = result.group(4)

        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()

        objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]

        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()

        objectId = req["gebouw"]["objectId"]

        req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()

        polygon = [req["geometriePolygoon"]["polygon"]]

        return polygon

In [4]:
'''Taking input address and assigning the same to both for_coordinates and polygon function '''

address=input("Enter the address ")

x , y = for_coordinates(address)

poly=polygon(address)

print ('x :', x ,'\ny :' , y )

Enter the address sint-pietersvliet 7, 2000 antwerpen
x : 152242.43 
y : 212849.5


In [5]:
''' dictionary making and finding the tile name having  coordinates '''

'''An empty dictionary'''
bounds_test = {}  

'''iterating through the number of sub divided tiff '''
for i in range (1, 641):
    
    path = "/home/becode/lidar/K_15/DTM/GeoTIFF/" + str(i) + ".tif"
    dtm = rio.open(path) 
    bounds_test[i] = dtm.bounds
    
'''to find the exact tile containing the coordinates'''
for keys,values in bounds_test.items():
    left = values[0]
    bottom = values[1]
    right = values[2]
    top = values[3]
    
    if (left < x < right) and (bottom < y < top):
        tile_number = keys
        print ('tile number :' ,tile_number)


tile number : 447


In [6]:
'''function for getting the required tile '''

def get_tile(tile_number, dsm_or_dtm):
    try:
        #Path to the tile file
        path = "/home/becode/lidar/K_15/" + str(dsm_or_dtm).upper() +"/GeoTIFF/" + str(tile_number)+ ".tif"
        tiff = rio.open(path)
        return tiff
    except:
        print('Check the path to the file containing the tiles.')


In [7]:

data_dsm = get_tile(tile_number,'dsm')
data_dtm = get_tile(tile_number,'dtm')



In [8]:

crop_dsm, crop_transform = mask(dataset=data_dsm, shapes=poly, crop=True, nodata=0, filled=True, indexes=1)
crop_dtm, crop_transform = mask(dataset=data_dtm, shapes=poly, crop=True, nodata=0, filled=True, indexes=1)

crop_chm = crop_dsm - crop_dtm


In [11]:
fig = go.Figure(data=go.Surface(z=crop_chm, colorscale='YlOrRd'))
fig.update_layout(title='3D Plot', autosize= True)

fig.show()